# Modelling Lending Club Loan Level Data
### by Jérôme Blanchet


# Abstract





# Table of Contents



# Part 1) 

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from scipy.stats import skew
from scipy.stats.stats import pearsonr
import seaborn as sns
from pandas import DataFrame
import time

In [5]:
dataA = pd.read_csv("C:/Users/jeblanch/Documents/LoanStats3a.csv")
dataB = pd.read_csv("C:/Users/jeblanch/Documents/LoanStats3b.csv")
dataC = pd.read_csv("C:/Users/jeblanch/Documents/LoanStats3c.csv")
dataD = pd.read_csv("C:/Users/jeblanch/Documents/LoanStats3d.csv")
dataE = pd.read_csv("C:/Users/jeblanch/Documents/LoanStats_2016Q1.csv")
dataF = pd.read_csv("C:/Users/jeblanch/Documents/LoanStats_2016Q2.csv")
dataG = pd.read_csv("C:/Users/jeblanch/Documents/LoanStats_2016Q3.csv")
dataH = pd.read_csv("C:/Users/jeblanch/Documents/LoanStats_2016Q4.csv")
dataI = pd.read_csv("C:/Users/jeblanch/Documents/PH.csv");

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
df = pd.concat((dataA, dataB, dataC, dataD, dataE, dataF, dataG, dataH), axis=0)
df.head(1)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [19]:
dataI.head(1)

,id,appl_fico_band,Last_FICO_BAND
0,54734,735-739,755-759


In [15]:
dataI = dataI.rename(columns={'\ufeffLOAN_ID': 'id'})

In [18]:
dataI.head(1)

,id,appl_fico_band,Last_FICO_BAND
0,54734,735-739,755-759


In [17]:
dff = pd.merge(df, dataI, on='id', how='left')

In [22]:
dff.head(1)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,appl_fico_band,Last_FICO_BAND
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,735-739,735-739


In [24]:
dff.shape

(1321792, 113)

In [23]:
dff.appl_fico_band.isnull().sum()

308816

In [25]:
dff.Last_FICO_BAND.isnull().sum()

308816

In [26]:
dff.loan_status.value_counts()

Current                                                928827
Fully Paid                                             289898
Charged Off                                             63666
Late (31-120 days)                                      14874
In Grace Period                                         11528
Issued                                                   6074
Late (16-30 days)                                        3967
Does not meet the credit policy. Status:Fully Paid       1988
Does not meet the credit policy. Status:Charged Off       761
Default                                                   206
Name: loan_status, dtype: int64

In [27]:
k = [i for i in dff.columns]
k

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_